In [1]:
import numpy as np
  
import autograd.numpy as anp       
from numpy import array
from numpy import reshape
import pandas as pd

from pymoo.model.problem import Problem
from pymoo.factory import get_problem, get_reference_directions

prob = get_problem("truss2D")
pf = prob.pareto_front()

class MyProblem(Problem):

    def __init__(self):
        super().__init__(n_var=3, n_obj=1, n_constr=0,type_var=anp.double, elementwise_evaluation=True )
        self.xl = anp.array([0.1,0.1,1])
        self.xu = anp.array([1.0,1.0,1100])

    def _evaluate(self, x, out, *args, **kwargs):
        global pf
        
        phi1 = pf[:,0]
        phi2 = pf[:,1]
        
        m=np.size(phi1,0)
        d=x[2]
        b=np.random.randn(2,1)
        b[0]=x[0]
        b[1]=x[1]
        b=b/np.amax(np.absolute(b))
        c=np.multiply((phi1**(b[0])),(phi2**(b[1])))
        c=np.sort(c,axis=0)
        rang=((c[m-1]-c[0])/d)
        split=np.arange(c[0], c[m-1],rang)
        split=np.append(split,(c[m-1]+1))  #1 --> can be any number greater other than 0
        N=np.full([d,m],np.nan)

        for i in range(d):
            for j in range(m):
                if split[i]<=c[j]<split[i+1]:
                    N[i][j]=c[j]


        count=np.zeros(d) 
        for i in range(d):
            for j in range(m):
                if np.any(~np.isnan(N[i][j])) and (count[i]<=m) :
                    count[i]=count[i]+1    

        C=0
        U=0
        sc=0
        Y=np.array([])
        flag=1
        for i in range(d):
            if count[i]<=(m/d):
                U=U+count[i]
                flag=1
            else:
                sc=sc+1
                Y=np.append(Y,i)
                if flag==1:
                    C=C+1
                flag=0

        
        Y=Y.astype(int)
        N1=np.empty([sc,m],dtype=float)
        N1[:,:]=np.nan
        for i in range(sc):
            for j in range(m):
                N1[i,j]=N[Y[i],j]
                        
        cv=np.zeros(sc)    
        cv=((np.nanstd(N1,axis=1))/np.nanmean(N1,axis=1))*100
        CV=0 
        for i in range(sc):
            CV=CV+cv[i]
    
        
        
        costfunc=C+U+CV  
        print(costfunc,C,U,d)
        out["F"] =  costfunc   

In [2]:
# if the first variable is an integer and the second a real value
mask = ["real", "real","int"]

# e.g. if first two are integers and third real it would be
# mask = ["int", "int", real"]
# for the purpose of plotting we limit us to two variables here

In [3]:
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.operators.mixed_variable_operator import MixedVariableSampling, MixedVariableMutation, MixedVariableCrossover

sampling = MixedVariableSampling(mask, {
    "real": get_sampling("real_random"),
    "int": get_sampling("int_random")
})

crossover = MixedVariableCrossover(mask, {
    "real": get_crossover("real_sbx", prob=0.9, eta=3.0),
    "int": get_crossover("int_sbx", prob=0.9, eta=3.0)
})

mutation = MixedVariableMutation(mask, {
    "real": get_mutation("real_pm", eta=3.0),
    "int": get_mutation("int_pm", eta=3.0)
})

In [4]:
import numpy as np

from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.factory import get_crossover, get_mutation, get_sampling
from pymoo.optimize import minimize

problem = MyProblem()

algorithm = GA(
    pop_size=100,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    eliminate_duplicates=True,
)

res = minimize(
    problem,
    algorithm,
    ('n_gen',100),
    seed=1,
    save_history=True
)

print("Best solution found: %s" % res.X)
print("Function value: %s" % res.F)
print("Constraint violation: %s" % res.CV)

d=res.X[2]
b[0]=res.X[0]
b[1]=res.X[1]

343.5901436310798 14 321.0 459
564.5383452507015 117 434.0 793
1100.0 0 1100.0 1
299.9041895899309 4 267.0 333
313.09591122394966 3 302.0 162
330.3654953160801 2 324.0 102
396.7782051011829 5 385.0 205
265.2407117302557 8 229.0 381
341.42013283177425 15 322.0 437
274.19945418794015 89 149.0 593
358.28142532458423 32 325.0 462
315.56993735715037 28 269.0 754
357.04580850480784 6 340.0 225
433.7269895622867 45 382.0 966
300.1306422435568 1 287.0 31
333.1346141823243 36 287.0 738
599.4751558500528 101 476.0 460
319.77196806878 91 201.0 615
469.6601240619496 5 453.0 155
370.8093973627167 4 363.0 218
776.0596111466117 184 580.0 881
487.12272864268016 51 431.0 1066
356.5628952490735 7 342.0 345
318.8415158859806 29 272.0 762
938.7496020316647 171 757.0 965
353.71561965571027 26 304.0 985
414.84432930463095 3 396.0 94
303.9808569720991 1 285.0 43
293.68527283289717 3 273.0 187
472.96001159153 58 412.0 966
325.07307233555855 1 312.0 109
354.86685769711494 15 334.0 464
898.9506862960534 157 736

325.6306243184987 11 297.0 481
448.18260768898915 55 390.0 913
295.5451247314481 1 264.0 40
524.3108330416825 9 503.0 219
293.49806092783336 4 276.0 223
300.7896422460009 1 283.0 166
412.6678161117838 43 362.0 926
296.31867012492836 35 254.0 635
448.7233325950201 2 428.0 57
430.89629623431887 17 412.0 536
366.7824564473017 9 354.0 339
341.45823142698293 33 296.0 787
293.91441398609186 4 264.0 316
368.20252211500446 44 320.0 766
328.9689642094792 1 316.0 100
1100.0 0 1100.0 1
307.40214679051064 6 276.0 341
293.1166116197466 1 268.0 113
286.59505130095977 3 265.0 229
290.37676279416485 23 241.0 710
313.4680194504667 11 288.0 441
363.40045721639547 13 339.0 517
311.78706472292527 9 276.0 493
320.7529999287566 6 298.0 332
275.1640690287332 5 239.0 293
329.5638240170942 1 321.0 40
298.6108115757174 24 249.0 736
300.41397493878054 1 279.0 37
326.82158777930823 7 312.0 314
312.9665370873451 23 262.0 825
377.4163514137737 31 329.0 994
326.77873157802065 26 275.0 866
309.58601283665485 10 280.0

350.94138927593724 5 340.0 258
267.9963993269212 8 228.0 389
148.1885672081957 17 130.0 650
322.9033313876666 1 314.0 76
381.6146182587264 10 366.0 322
456.8930998970374 13 436.0 343
401.79525058698056 1 395.0 44
348.7082886590218 1 343.0 30
345.6021664404466 44 297.0 709
295.3777014797861 1 275.0 39
495.1034112782847 86 408.0 975
100.52683448473256 1 99.0 281
531.6957249325442 38 483.0 490
332.4520658057075 21 310.0 320
97.3231863033712 1 96.0 166
320.71037788504657 26 270.0 828
304.5180252002579 1 283.0 34
332.69886959383166 1 329.0 33
340.1047194946165 3 332.0 149
205.88074426585223 20 185.0 412
419.77499888338 20 396.0 514
315.42662931773793 1 299.0 7
290.8465807815787 2 261.0 149
421.5083066910342 55 365.0 874
177.14196695039854 2 174.0 212
434.579559093819 3 420.0 108
96.01000804213766 1 95.0 146
396.4452781543968 1 387.0 49
96.64471587349576 1 95.0 89
619.7445822237728 45 566.0 533
267.46828714814666 25 224.0 599
94.03274022259023 1 93.0 289
401.9066807414522 4 392.0 209
304.478

396.6165973795424 21 368.0 439
406.02295803439415 74 331.0 815
139.52399285776073 16 123.0 1079
480.7292844289253 11 458.0 261
346.174226289564 1 344.0 5
286.465848462824 6 261.0 289
91.05892695679972 1 90.0 196
507.2049383428395 72 429.0 864
347.0810998483581 8 335.0 319
285.4839216285851 1 256.0 114
97.38336547249588 1 96.0 79
324.8433341506802 59 265.0 739
68.4584405255741 1 66.0 4
403.42972730414084 54 348.0 837
99.06279791526305 1 98.0 378
284.784064116855 35 248.0 634
80.27011317310458 1 79.0 351
1100.0 0 1100.0 1
126.46184055150388 18 108.0 1050
71.55564251727812 1 70.0 114
503.5922423785592 81 422.0 1027
225.48653362551138 1 223.0 6
97.63792116502356 1 96.0 230
478.10489411693624 32 435.0 466
93.62650783252994 1 92.0 42
413.62469044649924 13 392.0 321
436.85425520701574 21 410.0 485
367.1428858745269 1 361.0 30
365.13158140592395 19 345.0 330
276.05168730938965 6 267.0 288
71.39149743291753 1 70.0 113
309.07593293590173 15 291.0 396
400.1039002290534 65 325.0 698
85.13849628741

319.05547369051686 1 316.0 86
347.6969420583476 6 339.0 245
98.0926446077838 1 97.0 446
404.79441973211 1 392.0 71
448.83720932261593 54 392.0 944
353.52366793497174 6 346.0 181
548.2785586915986 14 523.0 274
86.12188746550649 1 85.0 519
222.7962212668347 2 219.0 162
72.40096043579639 1 71.0 206
238.79166639118066 29 206.0 564
99.29427095838629 1 98.0 853
144.98571779781042 1 143.0 117
328.1844977525291 1 318.0 67
556.3625038592364 78 473.0 932
265.52805901903145 24 222.0 598
341.7766211599422 3 333.0 160
181.31193233399108 1 179.0 38
374.6610585377435 18 354.0 465
290.48249409089135 32 257.0 396
223.094243609964 1 213.0 3
384.8398508462075 21 355.0 424
344.0653031638282 15 327.0 534
525.1359973028588 20 494.0 348
345.7296029169177 8 328.0 338
76.2190050485172 1 75.0 8
326.2644823593766 32 280.0 755
264.95698407185984 6 256.0 286
229.88782270618935 2 226.0 160
78.2494314641642 1 77.0 319
107.48571775663588 17 90.0 919
335.3311033192891 14 319.0 505
374.29771150836 12 361.0 265
357.7354

...

260.17307516135224 11 247.0 431
1100.0 0 1100.0 1
323.0874236436333 2 314.0 125
350.43200134016564 1 343.0 9
404.91178985992866 11 387.0 324
1100.0 0 1100.0 1
71.39007002489205 1 70.0 42
1100.0 0 1100.0 1
366.14797165492564 10 350.0 309
1100.0 0 1100.0 1
76.53678264332063 2 74.0 149
1100.0 0 1100.0 1
371.16530535428484 9 358.0 335
168.33059437801168 1 166.0 43
368.30535825468525 6 359.0 258
399.93440810252383 6 393.0 183
386.31963941736063 47 337.0 785
71.54361299599638 2 69.0 165
40.51444471294621 1 39.0 2
1100.0 0 1100.0 1
62.04129208711387 1 60.0 2
72.50078229106617 1 71.0 248
228.85722827475365 1 226.0 2
1100.0 0 1100.0 1
181.38372026799004 1 179.0 46
1100.0 0 1100.0 1
1100.0 0 1100.0 1
74.38099653600075 2 72.0 540
1100.0 0 1100.0 1
78.69745957683888 1 76.0 2
99.19692190833872 1 98.0 396
346.9703248802072 3 339.0 195
218.36864447872543 1 215.0 2
1100.0 0 1100.0 1
82.0431821132065 1 81.0 12
139.8576975259883 1 138.0 24
87.66984906815846 1 86.0 160
54.80282550764729 1 53.0 2
86.10190

316.55989692067396 1 314.0 51
1100.0 0 1100.0 1
1100.0 0 1100.0 1
220.0825498306575 1 218.0 2
358.7809577351513 6 348.0 246
44.50290362491488 1 43.0 2
1100.0 0 1100.0 1
1100.0 0 1100.0 1
1100.0 0 1100.0 1
346.42487889789896 1 342.0 105
359.4150953221736 3 349.0 141
324.92769813872144 3 318.0 198
1100.0 0 1100.0 1
293.4235653047786 46 237.0 571
333.93281209123325 1 324.0 133
1100.0 0 1100.0 1
1100.0 0 1100.0 1
76.28950819031289 1 75.0 63
378.3722527046959 2 369.0 94
283.5845681114571 13 264.0 370
70.48349883847696 1 69.0 78
1100.0 0 1100.0 1
1100.0 0 1100.0 1
1100.0 0 1100.0 1
1100.0 0 1100.0 1
179.37703538105353 3 175.0 231
1100.0 0 1100.0 1
75.30824398789561 1 74.0 153
281.2546389853008 5 273.0 225
210.85645600835025 1 206.0 2
359.24333131961555 2 356.0 73
74.37330825436722 2 72.0 835
83.14675136722073 1 82.0 221
45.52765647681061 1 44.0 2
1100.0 0 1100.0 1
359.30981515049496 2 356.0 100
1100.0 0 1100.0 1
327.0530783815711 3 320.0 144
98.13684845033949 1 97.0 401
1100.0 0 1100.0 1
40.

87.10181908902952 1 86.0 226
1100.0 0 1100.0 1
99.02877179514377 1 98.0 158
320.841922498669 2 315.0 104
1100.0 0 1100.0 1
1100.0 0 1100.0 1
1100.0 0 1100.0 1
1100.0 0 1100.0 1
1100.0 0 1100.0 1
87.28960333711395 1 86.0 74
98.12678097405139 1 97.0 135
76.287230502545 1 75.0 172
86.11472913898706 1 85.0 57
403.2808655901981 2 395.0 135
1100.0 0 1100.0 1
1100.0 0 1100.0 1
1100.0 0 1100.0 1
214.4232724362369 1 212.0 154
454.67466631747624 1 450.0 2
444.0297653055537 1 440.0 2
1100.0 0 1100.0 1
82.1761411256514 1 81.0 91
218.34771475927437 1 215.0 2
88.07636363585463 1 87.0 53
1100.0 0 1100.0 1
40.52160935139404 1 39.0 2
1100.0 0 1100.0 1
79.48560348235985 3 76.0 351
47.56850541024401 1 46.0 2
1100.0 0 1100.0 1
1100.0 0 1100.0 1
1100.0 0 1100.0 1
58.937009161746964 1 57.0 2
547.6720028255871 1 544.0 2
343.57791866965493 1 335.0 42
308.83636828797864 14 290.0 401
1100.0 0 1100.0 1
82.18121967476822 1 81.0 323
1100.0 0 1100.0 1
346.8182268005085 1 341.0 82
1100.0 0 1100.0 1
1100.0 0 1100.0 1

321.58259825021884 1 319.0 42
306.3968925544895 15 289.0 392
1100.0 0 1100.0 1
197.2958352322412 1 189.0 2
1100.0 0 1100.0 1
1100.0 0 1100.0 1
1100.0 0 1100.0 1
92.04181468368103 1 91.0 146
237.13023124295458 1 234.0 101
96.16189826069365 1 95.0 134
360.93121478883194 2 353.0 87
1100.0 0 1100.0 1
227.04897843466318 1 225.0 7
201.07815458243925 1 199.0 2
1100.0 0 1100.0 1
333.8502732217393 6 323.0 233
295.80216307882074 27 249.0 692
46.524978256961596 1 45.0 2
1100.0 0 1100.0 1
87.11185930073081 1 86.0 229
98.14119843400567 1 97.0 481
1100.0 0 1100.0 1
216.70470317556234 1 213.0 2
98.34902921807878 1 97.0 133
89.07146166070991 1 88.0 88
1100.0 0 1100.0 1
201.207994880893 1 194.0 2
195.53810862937343 1 193.0 141
463.1105444586583 54 406.0 946
62.07807382412888 1 60.0 2
1100.0 0 1100.0 1
1100.0 0 1100.0 1
97.43920656735023 1 96.0 90
1100.0 0 1100.0 1
361.5928227016724 4 353.0 204
419.0524285967852 1 410.0 2
193.50084410548342 1 191.0 9
398.52715152667275 6 390.0 275
320.5696613402732 1 31

NameError: name 'b' is not defined